In [64]:
#numeric: pandas and numpy
import numpy as np
import pandas as pd
import seaborn as sns

# graphics
%matplotlib inline 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

from sklearn import linear_model, svm
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_validate, GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score,zero_one_loss

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

## Read Data

In [65]:
name_features = []
with open('sensors-19-05524-s001/sensors-19-05524-s001_new/labels.txt', 'r') as file:
    content = file.read()
    cols = content.split('\n')
    name_features = cols[1:-2]   # remove first column and the last two
    
Y = []
X = []

with open('sensors-19-05524-s001/sensors-19-05524-s001_new/data.txt', 'r') as file:
    content = file.read()
    cols = content.split('\n')
    for c in cols[:-1]:
        data = c.split(',')
        X.append(data[1:-1])
        Y.append(data[-1])
        
# Convert String to Float
X = np.array(X).astype(float)
print(type(X[0][0]))

#Convert String to Int
Y = np.array(Y).astype(int)
print(type(Y[0]))
    
print(f"Features: {len(X[0])}")
print(f"Records in DB: {len(X)}")
print(f"Size Y: {len(Y)}")

<class 'numpy.float64'>
<class 'numpy.int64'>
Features: 533
Records in DB: 4480
Size Y: 4480


## Selection of Features (PCA)

In [66]:
data_features = pd.DataFrame(X, dtype='float')

corr_matrix = data_features.corr().abs()

In [67]:
def drop_features(ds, corr, threshold=0.8):
    # Select upper triangle of correlation matrix
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    return ds.drop(ds[to_drop], axis=1)

reduced_dataset = drop_features(data_features, corr_matrix)
reduced_dataset.head()

,0,1,4,5,8,11,13,14,15,18,...,445,447,448,451,454,456,485,488,489,495
0,-0.004125,0.254095,-0.538509,5.95534,-0.10937,-0.065212,0.001901,231.814,16.2733,0.374904,...,-4.85458,2.357070,9.48421,-15.5916,4.14944,-2.59232,16.0,798.68,0.63875,31.4247
1,0.031029,0.193761,0.781415,5.18794,-0.08737,-0.057924,0.002046,230.500,15.5536,0.397898,...,-6.90565,2.476570,11.24150,-16.9271,5.89258,-2.59483,20.0,798.68,0.68500,31.4359
2,0.015678,0.182336,0.881194,5.66530,-0.08037,-0.042190,0.002114,240.707,20.8210,0.725535,...,-5.94592,1.068740,4.24641,-14.9459,24.62180,-2.60259,24.0,798.68,0.68500,31.4478
3,0.014525,0.176636,1.024900,6.10968,-0.08037,-0.044888,0.002177,247.308,22.2275,0.267597,...,-6.18900,0.828304,4.28808,-14.6317,-28.60590,-2.61594,28.0,798.68,0.68375,31.4331
4,0.010349,0.179248,0.935697,5.83902,-0.08337,-0.041310,0.002217,247.744,20.1621,0.715390,...,-6.49062,0.988309,5.30232,-13.6923,-65.12220,-2.63103,32.0,798.68,0.71375,31.4347


In [68]:
X = reduced_dataset.to_numpy(dtype="float")
print(f"Features: {len(X[0])}")
print(f"Records in DB: {len(X)}")
print(f"Size Y: {len(Y)}")

Features: 103
Records in DB: 4480
Size Y: 4480


In [69]:
from sklearn.feature_selection import SelectKBest, f_classif

def get_best_x_features(df, dclass, num_features=50):

    df = (df - np.min(df))/(np.max(df) - np.min(df))
    
    k_bestfeatures = SelectKBest(score_func = f_classif)
    rankings = k_bestfeatures.fit(df, dclass)
    b=list(rankings.scores_)
    a=list(range(0,len(b)))

    sf = [g for _,g in sorted(zip(b,a))]
    sf=sf[len(a)-num_features:len(a)]
    sf=reversed(sf)
    inx=[]
    for chosen in sf:
        inx.append(chosen)
    
    dataset = pd.DataFrame(df, dtype='float')
    
    return dataset[inx].to_numpy(dtype='float')

In [70]:
X = get_best_x_features(X,Y)
print(f"Features: {len(X[0])}")
print(f"Records in DB: {len(X)}")
print(f"Size Y: {len(Y)}")

['order', 'index', 'column name', 'f-score']
Features: 50
Records in DB: 4480
Size Y: 4480


## Training Data

In [41]:
k_fold = StratifiedKFold(n_splits=4)

In [42]:
def processKFold(model, X, Y):
    
    scores = []
    loss = []
    
    for train_indexes, test_indexes in k_fold.split(X,Y):

        # maybe can be need normalize the data
        #scaler = StandardScaler()

        X_train, X_test = X[train_indexes], X[test_indexes]
        Y_train, Y_test = Y[train_indexes], Y[test_indexes]

        #X_train = scaler.fit_transform(X_train)
        #X_test = scaler.transform(X_test)

        model.fit(X_train, Y_train)

        Y_pred = model.predict(X_test)
        
        scores.append(accuracy_score(Y_test, Y_pred))
        loss.append(zero_one_loss(Y_test, Y_pred))
        
    return np.mean(scores), np.mean(loss)

In [5]:
# Multilayer feedforward neural network

# change number after
def mlp30(X, Y):
    
    ppnML = MLPClassifier(hidden_layer_sizes=(30),activation="relu",random_state=1)

    return processKFold(ppnML, X, Y)

In [6]:
# svm rbf and svm linear 

def svmRBF(X, Y):

    svmRBF = svm.SVC(kernel='rbf')
    
    return processKFold(svmRBF, X, Y)

In [7]:
def svmLinear(X, Y):

    svmLinear = svm.SVC(kernel='linear')
    
    return processKFold(svmLinear, X, Y)

In [8]:
acc_mpl, loss_mpl = mlp30(X, Y)

print(f"MPL:\nAcc: {acc_mpl}\nLoss: {loss_mpl}")

Mean acc mpl: (0.3236607142857143, 0.6763392857142857)


In [9]:
acc_svmRBF, loss_svmRBF = svmRBF(X, Y)

print(f"svmRBF:\nAcc: {acc_svmRBF}\nLoss: {loss_svmRBF}")

Mean acc svmRBF: (0.33816964285714285, 0.6618303571428571)


In [ ]:
acc_svmLinear, loss_svmLinear = svmLinear(X, Y)

print(f"svmLinear:\nAcc: {acc_svmLinear}\nLoss: {loss_svmLinear}")